# Project 6: NMF

Select a basic, explicit feedback dataset, many examples here:

<https://gist.github.com/entaroadun/1653794>

<https://github.com/caserec/Datasets-for-Recommender-Systems>

<https://github.com/RUCAIBox/RecSysDatasets>

And use the Surprise library:

<http://surpriselib.com/>

To implement a basic recommendation system. Many of those datasets are already loaded into the Surprise package to make this easy. You should tune and cross validate your system to select the best values for the # of latent dimensions, the regularization parameter, and any other hyperparameters.

Stretch Goal #1 (3 points)

Using an explicit feedback dataset, implement the NMF algorithm by hand, tune it via cross validation to select the # of latent dims and regularization parameter

Stretch goal #2 (3 points)

Using an implicit feedback dataset, some can be found here:

<https://cseweb.ucsd.edu/~jmcauley/datasets.html>

And implement the implicit feedback version of NMF. Evaluating performance will be harder however, so instead of doing a grid search and tuning, try to output the ratings explanations (i.e. use eqn 7 of the implicit feedback dataset).  Nobody has ever attempted this stretch goal.


In [2]:
from surprise import NMF, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [3]:
# Load the MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:5]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596')]

In [4]:
# Tune hyperparameters
param_grid = {
    'n_factors': [10, 20, 30], # number of latent factors
    'n_epochs': [10, 20, 30], # number of epochs
    'biased': [False] # use biases or not
}

gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(data)
best_params = gs.best_params['rmse']
print("Best parameters:", best_params)
print("Best RMSE:", gs.best_score['rmse'])

Best parameters: {'n_factors': 10, 'n_epochs': 30, 'biased': False}
Best RMSE: 1.0158539369859525


In [5]:
# Train the system
test_size = 0.2
trainset, testset = train_test_split(data, test_size=test_size)
algo = NMF(**best_params)
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
print("Test RMSE:", rmse)

RMSE: 1.0214
Test RMSE: 1.0213946848067066


In [6]:
# Recommend items for a specific user
user_id = '196'  # MovieLens user ID

all_items = set(iid for (_, iid, _, _) in data.raw_ratings)
rated_items = set(iid for (uid, iid, _, _) in data.raw_ratings if uid == str(user_id))
unrated_items = list(all_items - rated_items)
predictions = [(item, algo.predict(str(user_id), item).est) for item in unrated_items]
recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]

print(f"Top recommendations for user {user_id}:", [item for item, _ in recommendations])

Top recommendations for user 196: ['1639', '851', '814', '1175', '1449']


In [7]:
from surprise import Dataset, NMF, accuracy
from surprise.model_selection import GridSearchCV, train_test_split

class RecommendationSystem:
    def __init__(self, data: Dataset):
        """
        Initialize the recommendation system with a Surprise dataset.
        Args:
            data (surprise.Dataset): Surprise dataset containing user-item ratings.
        """
        self.data = data
        self.algo = None
        self.best_params = None

    def tune_hyperparameters(self, param_grid: dict, cv: int = 5):
        """
        Tune hyperparameters using GridSearchCV.
        Args:
            param_grid (dict): Dictionary of hyperparameter ranges for tuning.
            cv (int): Number of cross-validation folds.
        Returns:
            float: Best RMSE score from hyperparameter tuning.
        """
        gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=cv, n_jobs=-1)
        gs.fit(self.data)
        self.best_params = gs.best_params['rmse']
        return gs.best_score['rmse']

    def train(self, test_size: float = 0.2):
        """
        Train the recommendation system using the best hyperparameters.
        Args:
            test_size (float): Proportion of the dataset to use for testing.
        Returns:
            float: RMSE on the test set.
        """
        trainset, testset = train_test_split(self.data, test_size=test_size)
        self.algo = NMF(**self.best_params)
        self.algo.fit(trainset)
        predictions = self.algo.test(testset)
        rmse = accuracy.rmse(predictions)
        return rmse

    def recommend(self, user_id, n=10):
        """
        Recommend top-N items for a given user.
        Args:
            user_id (int): ID of the user.
            n (int): Number of recommendations to return.
        Returns:
            list: List of recommended item IDs.
        """
        all_items = set(iid for (_, iid, _, _) in self.data.raw_ratings)
        rated_items = set(iid for (uid, iid, _, _) in self.data.raw_ratings if uid == str(user_id))
        unrated_items = list(all_items - rated_items)

        predictions = [(item, self.algo.predict(str(user_id), item).est) for item in unrated_items]
        recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
        return [item for item, _ in recommendations]

In [8]:
# # Load HCAHPS Hospital data from Excel file
# df_hcahps = pd.read_excel('HCAHPS-Hospital.xlsx', usecols=['Facility ID', 'Patient Survey Star Rating'], nrows=100)
df_hcahps = pd.read_excel('HCAHPS-Hospital.xlsx')
df_hcahps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444447 entries, 0 to 444446
Data columns (total 22 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   Facility ID                            444447 non-null  object        
 1   Facility Name                          444447 non-null  object        
 2   Address                                444447 non-null  object        
 3   City/Town                              444447 non-null  object        
 4   State                                  444447 non-null  object        
 5   ZIP Code                               444447 non-null  int64         
 6   County/Parish                          444447 non-null  object        
 7   Telephone Number                       444447 non-null  object        
 8   HCAHPS Measure ID                      444447 non-null  object        
 9   HCAHPS Question                        444447 no

In [9]:
# Load the MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:5] # columns: user_id, item_id, rating, timestamp

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596')]

In [10]:
# Tune the recommendation system
recommender = RecommendationSystem(data)
param_grid = {
    'n_factors': [10, 20, 30], # number of latent factors
    'n_epochs': [10, 20, 30], # number of epochs
    'biased': [False] # use biases or not
}
recommender.tune_hyperparameters(param_grid)
print("Best parameters:", recommender.best_params)

Best parameters: {'n_factors': 10, 'n_epochs': 30, 'biased': False}


In [11]:
# Train the system and evaluate
rmse = recommender.train()
print("Test RMSE:", rmse)

RMSE: 1.0104
Test RMSE: 1.0103570105967794


In [12]:
# Recommend items for a specific user
user_id = '196'  # MovieLens user ID as a string
recommendations = recommender.recommend(user_id, n=5)
print(f"Top recommendations for user {user_id}:", recommendations)

Top recommendations for user 196: ['850', '1651', '868', '1450', '814']


## Stretch Goal 1: NMF by hand

In [13]:
def nmf_alr(R:np.ndarray, n_components:int, max_iter:int=100, tol:float=1e-4, reg:float=0.0):
    """
    Perform Non-Negative Matrix Factorization (NMF) using Alternating Linear Squares.
    
    Parameters:
    - R: np.ndarray
        The input matrix (m x n) to factorize.
    - n_components: int
        Number of latent dimensions (rank of the factorization).
    - max_iter: int
        Maximum number of iterations.
    - tol: float
        Convergence tolerance for reconstruction error.
    - reg: float
        Regularization parameter to avoid overfitting.
    
    Returns:
    - W: np.ndarray
        User feature matrix (m x n_components).
    - H: np.ndarray
        Item feature matrix (n_components x n).
    - R_hat: np.ndarray
        Reconstructed matrix (approximation of R).
    """
    # Initialize W and H with random non-negative values
    m, n = R.shape
    W = np.random.rand(m, n_components)
    H = np.random.rand(n_components, n)

    # Alternating optimization
    for iteration in range(max_iter):
        # Fix H, solve for W
        for i in range(m):
            H_transpose = np.dot(H, H.T) + reg * np.eye(n_components)
            W[i, :] = np.linalg.solve(H_transpose, np.dot(R[i, :], H.T))
            W[i, :] = np.maximum(W[i, :], 0)  # Ensure non-negativity
        
        # Fix W, solve for H
        for j in range(n):
            W_transpose = np.dot(W.T, W) + reg * np.eye(n_components)
            H[:, j] = np.linalg.solve(W_transpose, np.dot(W.T, R[:, j]))
            H[:, j] = np.maximum(H[:, j], 0)  # Ensure non-negativity
        
        # Compute reconstruction error
        R_hat = np.dot(W, H)
        error = np.linalg.norm(R - R_hat, 'fro')

        # Check for convergence
        if error < tol:
            break

    return W, H, R_hat


In [14]:
# Test input matrix
R = np.array([[5, 3, 0, 1],
              [4, 0, 0, 1],
              [1, 1, 0, 5],
              [1, 0, 0, 4],
              [0, 1, 5, 4]])

# Parameters
n_components = 2
max_iter = 500
tol = 1e-4
reg = 0.01

# Get Feature matrices and reconstructed matrix
W, H, R_hat = nmf_alr(R, n_components, max_iter, tol, reg)

# Print results
print("Original Matrix (R):")
print(R)
print("\nUser Feature Matrix (W):")
print(W)
print("\nItem Feature Matrix (H):")
print(H)
print("\nReconstructed Matrix (R_hat):")
print(R_hat)


Original Matrix (R):
[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]]

User Feature Matrix (W):
[[0.         5.23556019]
 [0.         3.47585079]
 [1.13361791 1.31367621]
 [0.88795745 0.98363352]
 [1.69754708 0.        ]]

Item Feature Matrix (H):
[[0.         0.38861719 1.75622767 3.09010109]
 [1.0083973  0.38160565 0.         0.28233911]]

Reconstructed Matrix (R_hat):
[[5.27952477 1.99791937 0.         1.47820343]
 [3.50503856 1.32640431 0.         0.98136864]
 [1.32470754 0.94184967 1.99089113 3.87389611]
 [0.99189338 0.72043564 1.55945543 3.02159649]
 [0.         0.65969597 2.98127915 5.24559209]]


In [15]:
# Load the Surprise built-in MovieLens-100k dataset
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()

# Convert the trainset to a user-item matrix
def build_user_item_matrix(trainset):
    R = np.zeros((trainset.n_users, trainset.n_items))
    for (uid, iid, rating) in trainset.all_ratings():
        R[int(uid), int(iid)] = rating
    return R

R = build_user_item_matrix(trainset)

# Apply NMF to the user-item matrix
n_components = 10  # Number of latent features
W, H, R_hat = nmf_alr(R, n_components=n_components, max_iter=100, tol=1e-4, reg=0.01)
R_hat


array([[6.72341260e-01, 5.95639220e-01, 8.08451509e-02, ...,
        2.12300976e-02, 2.12300976e-02, 2.12300976e-02],
       [3.32431114e-01, 1.25727991e+00, 4.12760656e-02, ...,
        1.50780448e-02, 1.50780448e-02, 1.50780448e-02],
       [1.43603012e-01, 3.15933317e-01, 1.00313395e-01, ...,
        1.86196042e-02, 1.86196042e-02, 1.86196042e-02],
       ...,
       [7.00483442e-01, 5.31157856e-01, 1.88152382e-02, ...,
        2.26009384e-02, 2.26009384e-02, 2.26009384e-02],
       [6.58500922e-01, 2.65107429e+00, 1.69687247e-04, ...,
        2.62181393e-03, 2.62181393e-03, 2.62181393e-03],
       [8.71384669e-02, 4.84739442e-01, 1.63240054e-02, ...,
        1.52432328e-03, 1.52432328e-03, 1.52432328e-03]])

In [16]:
def recommend(user_id, R, R_hat, trainset, top_n=5):
    """
    Recommend top N items for a given user.

    Parameters:
    - user_id: str, raw user ID.
    - R: np.ndarray, the original user-item matrix.
    - R_hat: np.ndarray, the reconstructed user-item matrix (predictions).
    - trainset: Trainset object, the Surprise trainset object.
    - top_n: int, number of recommendations to return.

    Returns:
    - recommendations: list of tuples (item_id, predicted_rating).
    """
    user_idx = trainset.to_inner_uid(user_id)  # Map raw user ID to internal ID
    scores = R_hat[user_idx, :]  # Predicted scores for all items
    # Get the list of items the user has already interacted with
    known_items = [iid for (iid, _) in trainset.ur[user_idx]]  # Trainset internal item IDs

    # Generate recommendations for items the user hasn't rated
    recommendations = [
        (trainset.to_raw_iid(iid), scores[iid])  # Map internal ID back to raw ID
        for iid in range(len(scores)) if iid not in known_items
    ]
    recommendations.sort(key=lambda x: x[1], reverse=True)  # Sort by predicted score
    return recommendations[:top_n]

In [17]:
# Test the recommendation system
user_id = '196'
top_recommendations = recommend(user_id, R, R_hat, trainset, top_n=5)

print(f"Top recommendations for user {user_id}:")
for item, score in top_recommendations:
    print(f"Item: {item}, Predicted Score: {score:.2f}")


Top recommendations for user 196:
Item: 275, Predicted Score: 1.31
Item: 14, Predicted Score: 1.24
Item: 83, Predicted Score: 1.12
Item: 137, Predicted Score: 1.07
Item: 283, Predicted Score: 1.00


In [18]:
def cross_validate_nmf(R, latent_dims, reg_params, n_splits=5, max_iter=100, tol=1e-4):
    """
    Perform cross-validation to tune the number of latent dimensions and regularization parameter.

    Parameters:
    - R: np.ndarray, the user-item matrix.
    - latent_dims: list of int, list of latent dimensions to test.
    - reg_params: list of float, list of regularization parameters to test.
    - n_splits: int, number of folds for cross-validation.
    - max_iter: int, maximum number of iterations for NMF.
    - tol: float, tolerance for convergence.

    Returns:
    - results: list of dict, each dict contains the hyperparameters and corresponding RMSE.
    """
    kf = KFold(n_splits=n_splits)
    results = []

    for n_components in latent_dims:
        for reg in reg_params:
            fold_rmse = []
            
            for train_idx, test_idx in kf.split(R):
                # Split data into train and test matrices
                R_train = np.zeros_like(R)
                R_test = np.zeros_like(R)
                for idx in train_idx:
                    R_train[idx] = R[idx]
                for idx in test_idx:
                    R_test[idx] = R[idx]

                # Apply NMF on the training data
                W, H, R_hat = nmf_alr(R_train, n_components, max_iter, tol, reg)

                # Evaluate RMSE on the test set
                mask = R_test > 0  # Only consider non-zero elements
                rmse = np.sqrt(np.mean((R_test[mask] - R_hat[mask]) ** 2))
                fold_rmse.append(rmse)

            # Average RMSE across folds
            avg_rmse = np.mean(fold_rmse)
            results.append({'n_components': n_components, 'reg': reg, 'rmse': avg_rmse})
            print(f"n_components={n_components}, reg={reg}, RMSE={avg_rmse:.4f}")

    return results

In [19]:
# Define hyperparameter grid
latent_dims = [5, 10, 20, 50]  # Number of latent features to test
reg_params = [0.0, 0.01, 0.1, 0.5]  # Regularization parameters to test

# Perform cross-validation
results = cross_validate_nmf(R, latent_dims, reg_params, n_splits=5, max_iter=100, tol=1e-4)

# Find the best hyperparameters
best_result = min(results, key=lambda x: x['rmse'])
print("\nBest Hyperparameters:")
print(f"n_components: {best_result['n_components']}, reg: {best_result['reg']}, RMSE: {best_result['rmse']:.4f}")


n_components=5, reg=0.0, RMSE=3.7095
n_components=5, reg=0.01, RMSE=3.7095
n_components=5, reg=0.1, RMSE=3.7095
n_components=5, reg=0.5, RMSE=3.7095
n_components=10, reg=0.0, RMSE=3.7095
n_components=10, reg=0.01, RMSE=3.7095
n_components=10, reg=0.1, RMSE=3.7095
n_components=10, reg=0.5, RMSE=3.7095
n_components=20, reg=0.0, RMSE=3.7095
n_components=20, reg=0.01, RMSE=3.7095
n_components=20, reg=0.1, RMSE=3.7095
n_components=20, reg=0.5, RMSE=3.7095
n_components=50, reg=0.0, RMSE=3.7095
n_components=50, reg=0.01, RMSE=3.7095
n_components=50, reg=0.1, RMSE=3.7095
n_components=50, reg=0.5, RMSE=3.7095

Best Hyperparameters:
n_components: 5, reg: 0.0, RMSE: 3.7095
